## LDA Preprocessing 1
# Artifact Removal and Stopword Selection
This notebook takes the clean JSON files for each article and does some preprocessing to obtain a text that we can analyze using LDA.

Specifically, we do:
* Artifact removal
* Stopword selection

In the next notebook we will do:
* Punctuation removal
* Lemmatization

In [32]:
import json
import re

We will also use some utility functions we defined in the `utils/` folder:
* `loadCorpusList(path)`: Loads the corpus as a list of `Article` objects (see `utils/Article.py`). This will allow us to save the clean text per document into the same JSON file with the metadata included.
* `saveCorpus(path)`: Saves the articles in JSON format in their current state. Useful when we want to append information to our clean JSON files.

In [33]:
import os
import sys 

# Jupyter Notebooks are not good at handling relative imports.
# Best solution (not great practice) is to add the project's path
# to the module loading paths of sys.

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.loaders import loadCorpusList, saveCorpus

## Loading the corpus

In [34]:
corpusPath = '../data/clean_json'

corpusList = loadCorpusList(corpusPath)

In [35]:
corpusList[0].title

'Los pecados del ateísmo'

We will only work with documents in Spanish. Hence, let's replace `corpusList` with only the articles that are in Spanish.

In [36]:
corpusList = [doc for doc in corpusList if doc.lang == 'es']

This leaves is with 554 articles.

In [37]:
len(corpusList)

554

(Este paso estaba en el otro notebook y siento que es importante pero no conozco bien la justificación.)

MGD: Creo que podemos borrar este paso. Lo hice para intentar quitar unas cosas que no se estaban quitando.

In [38]:
for doc in corpusList:
    doc.cleanText = doc.text.encode("utf8").decode("utf8")

## Artifact removal

There are some artifacts included in the text that are produced by HTML processing (or in the future because of how PDF files store text).

Let's start by removing numbers and some special characters such as newline characters (`\n`). We will keep normal punctuation for now as that might help SpaCy when we do lemmatization.

In [39]:
for doc in corpusList:
    doc.cleanText = re.sub('\d|\n',' ', doc.text)

We can detect some of these artifacts by looking for non-alphanumeric characters between alphanumeric characters (e.g. `"ar-gument"`, `"ar\xadgument"`).

In [40]:
artifacts = re.compile('\w+[^a-zA-ZáéíóúÁÉÍÓÚñÑüÜ\d\s:]\w+')

In [41]:
[re.findall(artifacts, doc.text) for doc in corpusList][0]

['reli\xadgión',
 'necesariamen\xadte',
 'in\xadterpretaciones',
 'rela\xadción',
 'tema.1',
 'diálo\xadgo',
 'dis\xadcusión',
 'creen\xadcia',
 'cf.Gal',
 '3.2',
 'pre\xaddicación',
 'subjeti\xadva',
 'fomen\xadtar',
 'so\xadbre',
 'sencilla\xadmente',
 'me\xaddicina',
 'meti\xadculosidad',
 'pros\xadperan',
 'conoci\xadmiento',
 'don\xadde',
 'esco\xadlástica',
 'razona\xadmiento',
 'lógico-matemática',
 'lógico-matemática',
 'lógico-matemática',
 'condescen\xaddientes',
 'cf.Comte',
 'controver\xadtible',
 'pue\xaddan',
 'fác\xadticas',
 'lógico-matemáticas',
 'tiem\xadpo',
 'comu\xadnicabilidad',
 'formal,3',
 'ago\xadta',
 'mé\xadtodo',
 'furio\xadso',
 '5.9',
 'pre\xadcepto',
 'C]uanto',
 'herma\xadnas',
 '10.21',
 '29-30',
 'pre\xadtendo',
 '1917-1980',
 'prin\xadcipalmente',
 'de\xadmostrar',
 'es\xadpecialmente',
 'cristia\xadnismo',
 'pre\xadcisamente',
 'argu\xadmento',
 'cons\xadtituye',
 'ha\xadber',
 'resenti\xadmiento',
 'nivela\xadción',
 '60-61',
 'Dios–todos',
 'ateís

One common artifact is the hex `\xad` for the soft hyphen which is used to break lines. We can remove it easily.

In [42]:
for doc in corpusList:
    doc.cleanText = re.sub('\\xad','', doc.cleanText)
    doc.cleanText = doc.cleanText.replace(u"\xa0", "")

We can save the corpus for now.

In [43]:
saveCorpus('../data/clean_json', corpusList)

## Stopword Removal
Stopword removal is perhaps the most difficult part of preprocessing. There are two challenges to meet:
* Some stopword lists such as the one included in NLTK for Spanish are too weak and do not filter many stopwords.
* Other stopword lists are too inclusive and can eliminate words that are meaningful in philosophy (e.g. 'verdadero', true). 
It is important to note that stopwords are very context-sensitive. A word in one context may provide little meaning (hence counting as a stopword) while in other contexts it may provide lots of information.

To tackle these challenges, we will first to an initial filtering with NTLK's list. This will leave many stopwords in the text, but will reduce the size of each text considerably. Then we will compare the text with a stronger list of stopwords (source). We will see which words are both the text and the stronger stopwords list. We will inspect these lists manually and extract a list of protected words. We will iterate over this process a number of times. Once we have a robust list of protected words, we will concatenate NLTK's stopwords list with the stronger one and eliminate the protected words from it. This will provide a final (hopefully middle ground) stopword list with which to continue.

In [44]:
from nltk.corpus import stopwords as nltk_stopwords

stopwords_weak = nltk_stopwords.words('spanish')

In [45]:
import requests

r = requests.get('https://raw.githubusercontent.com/stopwords-iso/stopwords-es/master/stopwords-es.txt')
stopwords_strong = r.text.split('\n')

In [46]:
docWords = []
for doc in corpusList:
    docWords += [word for word in re.findall('\w+', doc.cleanText) if word not in stopwords_weak]

In [47]:
docWords[:10]

['Introducción',
 'Quisiera',
 'comenzar',
 'disertación',
 'anécdota',
 'puede',
 'ser',
 'significativa',
 'tema',
 'convoca']

In [48]:
from collections import Counter

docs_and_stopwords = Counter([word for word in docWords if word in stopwords_strong])

In [49]:
docs_and_stopwords.most_common(10)

[('ser', 11710),
 ('puede', 7425),
 ('si', 6662),
 ('sino', 6335),
 ('mismo', 5462),
 ('bien', 5069),
 ('modo', 4206),
 ('decir', 4035),
 ('manera', 3754),
 ('solo', 3667)]

Already in the first 10 most common words in both the documents and the strong list of stopwords we find words that in philosophy are quite meaningful:
* 'ser': being
* 'bien': good
* 'modo': mode

We will start saving those words and eliminating them from the stronger list of stopwords. Then we will repeat the process of selecting the words that are in both lists and see which words are common. By iterating over this process a couple of times, we will get a list of protected words.

In [50]:
protectedWords = [
    'ser',
    'bien',
    'modo'
]

In [51]:
stopwords_strong = [word for word in stopwords_strong if word not in protectedWords]
docs_and_stopwords = Counter([word for word in docWords if word in stopwords_strong])

In [52]:
docs_and_stopwords.most_common(100)

[('puede', 7425),
 ('si', 6662),
 ('sino', 6335),
 ('mismo', 5462),
 ('decir', 4035),
 ('manera', 3754),
 ('solo', 3667),
 ('parte', 3640),
 ('dos', 3522),
 ('tal', 3423),
 ('así', 3160),
 ('pues', 3137),
 ('lugar', 3111),
 ('hace', 2895),
 ('debe', 2793),
 ('posible', 2769),
 ('vez', 2749),
 ('misma', 2709),
 ('hecho', 2688),
 ('embargo', 2585),
 ('entonces', 2476),
 ('poder', 2453),
 ('según', 2359),
 ('tiempo', 2304),
 ('ejemplo', 2302),
 ('ello', 2290),
 ('verdad', 2247),
 ('respecto', 2236),
 ('cosas', 2138),
 ('cada', 2128),
 ('partir', 2125),
 ('pueden', 2071),
 ('parece', 2060),
 ('cuanto', 2037),
 ('toda', 2008),
 ('b', 1992),
 ('aquí', 1958),
 ('sólo', 1942),
 ('cómo', 1941),
 ('hacer', 1935),
 ('primera', 1898),
 ('menos', 1895),
 ('general', 1852),
 ('siempre', 1844),
 ('trata', 1790),
 ('podría', 1780),
 ('propia', 1688),
 ('da', 1667),
 ('propio', 1604),
 ('saber', 1564),
 ('fin', 1561),
 ('tener', 1537),
 ('primer', 1502),
 ('sido', 1471),
 ('aunque', 1460),
 ('segundo',

In [53]:
protectedWords += [
    'parte',
    'posible',
    'lugar',
    'hecho',
    'poder',
    'verdad',
    'cosas',
    'general',
    'fin',
    'trabajo',
    'cierto',
    'uso',
    'dado',
    'diferentes',
    'verdadero',
    'verdadera',
    'existe',
    'valor',
    'realizar',
    'existen',
    'conocer',
    'diferente'
]

protectedWords = list(set(protectedWords))

After a couple of times to make the process less complex, once we are sure of a set of words, we can eliminate those from the list of document words and go back and repeat the process a couple more times.

In [54]:
stopwordsToRemove = [word[0] for word in docs_and_stopwords.most_common(100)]
docWords = [word for word in docWords if word not in stopwordsToRemove]

### Removing stopwords in English

Given that most of the articles have abstracts in English, some of the usual stopwords in English are appearing frequently in our documents. Thus, we will append the NLTK-generated list of English stopwords.

In [55]:
english_stopwords = nltk_stopwords.words("english")

TODO: should we be filtering these just like we filter the Spanish ones?

### Other stopwords custom to our corpus

There are some other stopwords that we would like to include, but that have not been taken into account in the previous processes. These are:

In [56]:
custom_stopwords = [
    "cf",
    "no obstante",
    "sin embargo",
    "por ejemplo",
    "es decir",
    "ak",
    "krv"
]

Once we have a robust set of words we can save both the final stopword list and the protected words list.

In [57]:
stopwords_final = list(set(
    stopwords_weak + stopwords_strong + english_stopwords + custom_stopwords
))
with open('wordlists/stopwords.txt', 'w') as fp:
    fp.write('\n'.join(stopwords_final))

with open('wordlists/protectedWords.txt', 'w') as fp:
    fp.write('\n'.join(protectedWords))